https://projecteuler.net/problem=207

$$4^t = 2^t + k$$

Let $q:=2^t$. Then we obtain a quadradic equation $q^2 - q - k = 0$, with the only allowed positive $q=\frac{1}{2} (1 + \sqrt{1 + 4k})$.

For this value to be an integer we require that $(1 + \sqrt{1 + 4k})$ is an even integer; hence, $1+4k$ should be a perfect square of an odd number:
$$
1 + 4k = (2n + 1)^2,
$$
from where we get that $k = n(n+1)$.

Given $k\leq m$ we obtain the condition on $n$:
$$
\begin{align*}
& n(n+1) \leq m \\
& 1 \leq n \leq \left\lfloor\frac{\sqrt{1+4m} - 1}{2}\right\rfloor =: T(m).
\end{align*}
$$
Here $T(m)$ counts the number of integer partitions with $k\leq m$.

Now, let us count the **perfect** integer partitions with $k \leq m$. For this, we require that $q$ is a power of two.

Since perfect integer partitions must be integer partitions, we take $k = n(n+1)$.
$$
q = \frac{1}{2} (1 + \sqrt{1 + 4k}) |_{k = n(n+1)} = 1 + n.
$$
Hence, $n = 2^l - 1$, and then $k = 2^l (2^l - 1)$.

Now, in order to to count the perfect integer partitions with $k \leq m$, we solve
$$
\begin{align*}
&2^l (2^l - 1) \leq m \\
& 1 \leq l \leq \left\lfloor \log_2{(1 + \sqrt{1 + 4m})} - 1 \right\rfloor =: B(m),
\end{align*}
$$
with $B(m)$ — the number of perfect integer partitions with $k\leq m$.

Then,
$$
P(m) = \frac{T(m)}{B(m)}.
$$

In [1]:
import math


def portion_perfect_partitions(M: int) -> float:
    return math.floor(math.log2(1 + math.sqrt(1 + 4 * M)) - 1) / math.floor(
        (math.sqrt(1 + 4 * M) - 1) / 2
    )


M_VALUES = [5, 10, 15, 20, 25, 30, 180, 185]
TRUE_P_VALUES = [1, 1 / 2, 2 / 3, 1 / 2, 1 / 2, 2 / 5, 1 / 4, 3 / 13]

for m_v, p_true in zip(M_VALUES, TRUE_P_VALUES):
    p_formula = portion_perfect_partitions(m_v)
    print(m_v, math.isclose(p_formula, p_true), p_formula, p_true)

5 True 1.0 1
10 True 0.5 0.5
15 True 0.6666666666666666 0.6666666666666666
20 True 0.5 0.5
25 True 0.5 0.5
30 True 0.4 0.4
180 True 0.25 0.25
185 True 0.23076923076923078 0.23076923076923078


The brute force approach would fail now since $P(m)$ decreases **very** slowly.

Let
$$
P^*(n) = P(n(n+1)) = \frac{\left\lfloor \log_2{(1 + n)} \right\rfloor}{n}.
$$

We could already run a brute force on this (try values of $n\in\mathbb{N}$) until the condition is satisfied, but there is a better way!

In order to get the sense on the size of $n$, we substitute further $n = 2^l - 1$:
$$
P^{**}(l) = P^*(2^l - 1) = \frac{l}{2^l - 1}.
$$
Here, one could use Lambert functions ($W(x) := f^{-1}(x)$ for $f(x)=xe^x$), or just brute force this and find that the smallest $l^*$ such that $P^{**}(l^*) < 1/12345$ is $l^* = 18$.

In [2]:
l_star = 1
while l_star / (2**l_star - 1) >= 1 / 12345:
    l_star += 1
print(l_star)

18


Then, 
$$
n^*\in[2^{17}-1, 2^{18}-1]=:I,
$$
where $n^*$ is the smallest $n$ such that $P^*(n) < 1/12345$.

Let is now find the only zero of the function
$$
F(n) := 1/12345 - P^*(n)
$$
on $I$ using interval bisection.

In [3]:
def p_star(n: int) -> float:
    return math.floor(math.log2(1 + n)) / n


def f(n: int) -> float:
    return 1 / 12345 - p_star(n)


def find_n(a: int, b: int) -> int:
    mid = (a + b) // 2
    mid_val = f(mid)
    if mid_val == 0 or a == b:
        return mid
    elif mid_val > 0:
        return find_n(a, mid)
    else:
        return find_n(mid, b)


In [4]:
n_star = find_n(2**17 - 1, 2**18 - 1) + 1
n_star

209866

In [ ]:
# f(n_star) > 0 and f(n_star - 1) <= 0
f(n_star - 2), f(n_star - 1), f(n_star)

(-3.8598547270993645e-10, 0.0, 3.8598179432367333e-10)

Then, $m^* = n^* (n^* + 1)$.

In [6]:
m_star = n_star * (n_star + 1)
m_star

44043947822